In [14]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split


def cargar_y_procesar_datos(path):
    """
    Carga y procesa los datos.

    Args:
    - path: Ruta del archivo CSV con los datos.

    Returns:
    - housing: DataFrame con los datos procesados.
    - null_rows_idx: Índices de las filas con valores nulos.
    """
    housing = pd.read_csv(path)

    # Generación de conjuntos de entrenamiento y prueba mediante muestreo estratificado por ingreso medio
    train_set, test_set = train_test_split(
        housing,
        test_size=0.2,
        stratify=pd.cut(
            housing["median_income"],
            bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
            labels=[1, 2, 3, 4, 5],
        ),
    )

    # Procesamiento de datos
    housing = train_set.drop(
        "median_house_value", axis=1
    )  # Eliminamos la columna de la variable dependiente

    # Identificación de valores no disponibles
    null_rows_idx = housing.isnull().any(
        axis=1
    )  # Índices de las filas con valores nulos

    return housing, null_rows_idx


def ejecutar_SimpleImputer(housing):
    """
    Ejecuta SimpleImputer para imputar los valores nulos.

    Args:
    - housing: DataFrame con los datos.
    - null_rows_idx: Índices de las filas con valores nulos.

    Returns:
    - housing_imputed: DataFrame con los valores nulos imputados.
    """
    # Creamos una instancia de `SimpleImputer`
    # para imputar los valores nulos con la mediana
    imputer = SimpleImputer(strategy="median")

    # Seleccionamos las columnas numéricas
    housing_num = housing.select_dtypes(include=[np.number])

    # Calculamos la mediana de cada columna numérica
    imputer.fit(housing_num)

    # Reemplazamos los valores nulos por la mediana
    housing_imputed = pd.DataFrame(
        imputer.transform(housing_num),
        columns=housing_num.columns,
        index=housing_num.index,
    )

    return housing_imputed


def ejecutar_KNNImputer(housing):
    """
    Ejecuta KNNImputer para imputar los valores nulos.

    Args:
    - housing: DataFrame con los datos.
    - null_rows_idx: Índices de las filas con valores nulos.

    Returns:
    - housing_imputed: DataFrame con los valores nulos imputados.
    """
    # Creamos una instancia de `KNNImputer`
    # para imputar los valores nulos con la estrategia de vecinos más cercanos
    imputer = KNNImputer(
        n_neighbors=5
    )  # Puedes ajustar el número de vecinos según tus necesidades

    # Seleccionamos las columnas numéricas
    housing_num = housing.select_dtypes(include=[np.number])

    # Imputamos los valores nulos utilizando KNN
    housing_imputed = pd.DataFrame(
        imputer.fit_transform(housing_num),
        columns=housing_num.columns,
        index=housing_num.index,
    )

    return housing_imputed

In [15]:
# Ruta del archivo CSV con los datos
archivo_datos = "./data/housing.csv"

# Cargar y procesar los datos
housing, null_rows_idx = cargar_y_procesar_datos(archivo_datos)

# Ejecutar SimpleImputer
housing_imputed_simple = ejecutar_SimpleImputer(housing)

# Ejecutar KNNImputer
housing_imputed_knn = ejecutar_KNNImputer(housing)

In [16]:
# Mostrar los primeros registros del DataFrame con valores nulos imputados
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
print("- \t\tMostramos los primeros registros del DataFrame transpuesto\t\t -")
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
# Mostramos los primeros registros del DataFrame transpuesto
print(housing.head().T)

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
- 		Mostramos los primeros registros del DataFrame transpuesto		 -
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                        20367   9197       9728    13513      4972 
longitude             -118.94 -119.65    -121.65 -117.33    -118.28
latitude                34.16   37.09      36.85   34.12       34.0
housing_median_age        3.0    17.0       20.0    38.0       38.0
total_rooms            1170.0  1280.0     2606.0  1703.0     3335.0
total_bedrooms          148.0   254.0      424.0   385.0      921.0
population              493.0   707.0     1361.0  1356.0     3612.0
households              142.0   267.0      426.0   363.0      887.0
median_income          8.0428    3.55     4.5787  2.0391      2.125
ocean_proximity     <1H OCEAN  INLAND  <1H OCEAN  INLAND  <1H OCEAN


In [17]:
# Mostrar los primeros registros del DataFrame con valores nulos imputados
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
print("- \t\tEjemplo de valores nulos imputados con SimpleImputer:\t\t -")
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
print(housing_imputed_simple.loc[null_rows_idx].head())

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
- 		Ejemplo de valores nulos imputados con SimpleImputer:		 -
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
2578     -124.06     40.86                34.0       4183.0           435.0   
9942     -122.23     38.17                45.0        350.0           435.0   
10761    -117.87     33.62                 8.0       1266.0           435.0   
20372    -118.88     34.17                15.0       4260.0           435.0   
3529     -118.53     34.26                18.0       3674.0           435.0   

       population  households  median_income  
2578       1891.0       669.0         3.2216  
9942        225.0        72.0         1.8942  
10761       375.0       183.0         9.8020  
20372      1701.0       669.0         5.1033  
3529       1590.0       550.0         8.1760  


In [18]:
# Mostrar los primeros registros del DataFrame con valores nulos imputados
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
print("- \t\tEjemplo de valores nulos imputados con KNNImputer:\t\t -")
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
print(housing_imputed_knn.loc[null_rows_idx].head())

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
- 		Ejemplo de valores nulos imputados con KNNImputer:		 -
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
2578     -124.06     40.86                34.0       4183.0           699.2   
9942     -122.23     38.17                45.0        350.0            77.2   
10761    -117.87     33.62                 8.0       1266.0           190.6   
20372    -118.88     34.17                15.0       4260.0           651.8   
3529     -118.53     34.26                18.0       3674.0           569.4   

       population  households  median_income  
2578       1891.0       669.0         3.2216  
9942        225.0        72.0         1.8942  
10761       375.0       183.0         9.8020  
20372      1701.0       669.0         5.1033  
3529       1590.0       550.0         8.1760  
